# Advanced Techniques for Text Generation with LangChain
Demonstrating the use of LangChain for complex text generation tasks.

## Why LangChain?
- Abstraction over multiple LLM APIs (GPT, LLaMA, Claude, etc.)
- Managing complex multi-step workflows with agents

## Installing LangChain and OpenAI
Make sure to install necessary dependencies.
```
pip install langchain openai
```

## Setting Up API Keys
Store API keys securely using environment variables.

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

## Loading a Chat Model
Example: Loading an OpenAI chat model with LangChain.

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Creating an Agent
Define an agent with specific tools and a task-focused context.

## Loading a Model Provider
In this example, we'll use OpenAI's GPT model.

In [ ]:
chat = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Managing API Keys (Outside Colab)

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

## Example: Loading OpenAI GPT model

```python
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
```

## Creating a Joke Generator
System message and human prompt for joke generation.

In [ ]:
messages = [
    SystemMessage(content="You are a helpful assistant that tells jokes."),
    HumanMessage(content="Tell me a joke about software engineers.")
]
response = chat.invoke(messages)
print(response.content)

## Streaming Chat Models
Demonstrating how to stream responses.

In [ ]:
for chunk in chat.stream(messages):
    print(chunk)

## Using Prompt Templates for Dynamic Prompts

In [ ]:
from langchain.prompts import PromptTemplate

template = """
You are a creative consultant brainstorming names for businesses.
You must follow the following principles:
{principles}
Please generate a numerical list of five catchy names for a start-up in the {industry} industry that deals with {context}.
"""
system_prompt = SystemMessagePromptTemplate.from_template(template)

## Batch API Requests with `.batch()`

In [ ]:
response = chat.batch(messages=[messages, messages])
print(response)

## Asynchronous API Requests

In [ ]:
async def async_invoke():
    result = await chat.ainvoke(messages)
    return result

## Pydantic Model for Structured Output

In [ ]:
from pydantic import BaseModel, Field

class BusinessName(BaseModel):
    name: str
    rating_score: float

In [ ]:
parser = PydanticOutputParser(pydantic_model=BusinessName)

## Chaining with LangChain

In [ ]:
from langchain.prompts import SystemMessagePromptTemplate
from langchain.chains import SimpleChain

template = """
You are an expert financial assistant. Categorize the following transaction description.

Transaction: {transaction_description}

Provide the transaction_type and transaction_category.

{format_instructions}
"""
system_prompt = SystemMessagePromptTemplate.from_template(template)

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["transaction_description"],
    template=template,
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

## Handling Large Text Chunks with Text Splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
docs = text_splitter.create_documents(["Sample text"])

## Summarization Chain (Map-Reduce)

In [ ]:
from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm=chat, chain_type="map_reduce")
summary = chain.invoke(docs)
print(summary['output_text'])

## Function Calling with LangChain

In [ ]:
import json

def schedule_meeting(date, time, attendees):
    return {
        "event_id": "1234",
        "status": "Meeting scheduled successfully!",
        "date": date,
        "time": time,
        "attendees": attendees
    }

functions = [
    {
        "type": "function",
        "function": {
            "type": "object",
            "name": "schedule_meeting",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "format": "date"},
                    "time": {"type": "string", "format": "time"},
                    "attendees": {"type": "array", "items": {"type": "string"}},
                },
                "required": ["date", "time", "attendees"],
            },
        },
    }
]

## Example: Scheduling a Meeting with OpenAI and LangChain

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

messages = [
    {"role": "user", "content": "Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob."}
]

response = client.chat.completions.create(model="gpt-3.5-turbo-1106", messages=messages, tools=functions)

## Handling Function Calls in Responses

In [ ]:
if response.choices[0].message.tool_calls:
    first_tool_call = response.tool_calls[0]
    function_name = first_tool_call.function.name
    function_args = json.loads(first_tool_call.function.arguments)
    function_response = schedule_meeting(**function_args)

    messages.append(
        {"role": "function", "name": "schedule_meeting", "content": json.dumps(function_response)}
    )

## Text Summarization Using Map-Reduce Chain

In [ ]:
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import pandas as pd

df = pd.DataFrame({"scenes": ["Scene 1 text", "Scene 2 text"]})
all_text = "\n".join(df.scenes.tolist())

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=1500, chunk_overlap=200)
docs = text_splitter.create_documents([all_text])

chain = load_summarize_chain(llm=chat, chain_type="map_reduce")
summary = chain.invoke(docs)
print(summary['output_text'])

## Final Example: Query Planning with LCEL

In [ ]:
from pydantic import BaseModel, Field

class Query(BaseModel):
    id: int
    question: str
    dependencies: list = Field(default_factory=list)

class QueryPlan(BaseModel):
    query_graph: list

In [ ]:
parser = PydanticOutputParser(pydantic_object=QueryPlan)
template = """Generate a query plan for the following task: {query}
Return the graph in the following format: {format_instructions}
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = PromptTemplate.from_messages([system_message_prompt])

In [ ]:
chain = chat_prompt | chat | parser

result = chain.invoke({
    "query": "I want to get results from my database, find the average age of top 10 customers, and send an email to John.",
    "format_instructions": parser.get_format_instructions()
})

print(result.query_graph)